# vLLM 

vLLM은 대규모 언어 모델의 추론 성능을 최적화하는 라이브러리입니다.

**주요 특징:**

- **PagedAttention**: 메모리 효율적인 attention 메커니즘
- **Continuous Batching**: 동적 배치 처리로 처리량 향상
- **Fast Sampling**: 효율적인 토큰 생성
- **OpenAI Compatible API**: OpenAI API와 호환되는 서빙 인터페이스

**성능 향상:**

- HuggingFace Transformers 대비 최대 24배 빠른 처리량
- GPU 메모리 사용량 최적화
- 높은 동시 요청 처리 능력

## 환경설정

In [1]:
!pip install langchain langchain-community langchain_huggingface transformers peft accelerate vllm chromadb pypdf sentence-transformers huggingface_hub

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.4/414.4 MB 85.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 239.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 81.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 180.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/

In [2]:
import os
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema.runnable import RunnablePassthrough

INFO 09-02 01:47:05 [__init__.py:241] Automatically detected platform cuda.


In [3]:
pdf_path = '[GIP] 2025 국가별 ICT 시장동향_보고서_미국.pdf'
loader = PyPDFLoader(pdf_path)
documents = loader.load()
print(f"PDF에서 추출된 문서 수: {len(documents)}")
print(f"첫 번째 문서 샘플: {documents[0].page_content[:200]}")

PDF에서 추출된 문서 수: 31
첫 번째 문서 샘플: 미국


In [4]:
# 텍스트 청킹
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
chunks = text_splitter.split_documents(documents)
print(f"청크 수: {len(chunks)}")

청크 수: 60


In [5]:
# 길이가 10 이하인 문서 필터링
filtered_chunks = [chunk for chunk in chunks if len(chunk.page_content) > 10]
print(f"필터링 전 청크 수: {len(chunks)}")
print(f"필터링 후 청크 수: {len(filtered_chunks)}")

# 필터링된 청크가 있으면 제거된 내용 샘플 출력
if len(chunks) != len(filtered_chunks):
    removed = [chunk for chunk in chunks if len(chunk.page_content) <= 10]
    print(f"제거된 청크 샘플 ({len(removed)}개): {removed}")
    print('--')

chunks = filtered_chunks
print(f"첫 번째 청크 샘플: {chunks[0].page_content}")

필터링 전 청크 수: 60
필터링 후 청크 수: 59
제거된 청크 샘플 (1개): [Document(metadata={'producer': 'Hancom PDF 1.3.0.546', 'creator': 'Hwp 2020 11.0.0.8362', 'creationdate': '2025-04-04T14:14:13+09:00', 'author': 'rnc', 'moddate': '2025-04-04T14:14:13+09:00', 'pdfversion': '1.4', 'source': '[GIP] 2025 국가별 ICT 시장동향_보고서_미국.pdf', 'total_pages': 31, 'page': 0, 'page_label': '1'}, page_content='미국')]
--
첫 번째 청크 샘플: Ⅰ ICT 국가 산업 현황  04(*) SUMMARY1. 국가 개황2. ICT 정부 기관3. ICT 주요 정책4. ICT 주요 법령 및 규제5. ICT 주요 기업6. 한국 협력 및 국내기업 진출 사례Ⅱ ICT 주요 동향  17(*) SUMMARY1. 인공지능2. 블록체인3. 클라우드 컴퓨팅4. 기업용 솔루션(SW)5. 빅데이터 Ⅲ 스타트업 생태계 24(*) SUMMARY1. 코그니션 AI(Cognition AI)2. 모나드 랩스(Monad Labs)3. 사이애라(Cyera)4. 플래드(Plaid)5. 웨이모(Waymo)※ 참고 문헌


## Indexing
- 임베딩 모델 로드
- 벡터스토어 구축

In [6]:
# 임베딩 모델
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'} # GPU 사용
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
vectorstore = Chroma.from_documents(chunks, embeddings)
print(f"벡터 DB가 생성되었습니다.")

벡터 DB가 생성되었습니다.


In [8]:
# 검색 테스트
query = "미국의 ICT 정책의 주요 내용은 무엇인가요?"
docs = vectorstore.similarity_search(query, k=3)
print(f"검색된 문서 수: {len(docs)}")
print(f"첫 번째 검색 결과: {docs[0].page_content[:500]}")

검색된 문서 수: 3
첫 번째 검색 결과: 국가별 ICT 시장동향 93. ICT 주요 정책① 미국 인공지능 리더십 장벽 제거* Federal Register, Removing Barriers to American Leadership in Artificial Intelligence, 2025.01.23.
n(주요  내용) 미국  AI 발전  저해 정 책 폐지  및 글로 벌 리더 십 강화  행동 계 획 추진-2025년 1월 23일 백악관에서 발표한 행정명령*으로, 이전 행정부의 행정명령 14110(안전하고 신뢰할 수 있는 AI 개발 및 사용)을 철회하고 해당 명령에 따른 정책과 지침 재검토 지시-AI 시스템에서 이념적 편향과 사회적 의제 배제를 강조하고, 경제 경쟁력과 국가 안보 증진에 초점을 맞춘 정책 방향 설정n(배경) 이전 행정부의 AI 정책 보다 유연한 혁신 환경 조성 필요성 대두-기존 AI 정책이 과도한 규제와 특정 가치 지향적 접근으로 기술 혁신과 경쟁력을 저해한다는 산업계 및 정치적 비판 증가-자유 시장, 세계적 수


## Retrieval-Generation

### Fine-tuned Model from local
- 병합후 vLLM모델 객체로 생성한다.

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model = AutoPeftModelForCausalLM.from_pretrained(
    "llama3-8b-rag-ko/checkpoint-846",
    device_map="auto"
)
# LoRA 등 PEFT 어댑터를 base 모델에 합쳐서 "단일 모델"로 만듭니다.
merged_model = peft_model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained("llama3-8b-rag-ko")  # 베이스 모델 토크나이저

# Hugging Face 형식으로 새로운 경로에 저장 (vLLM에서 로딩할 목적)
save_path = "./llama3-8b-rag-ko-full-merged"
merged_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


In [ ]:
# 원본 VLLM 라이브러리 직접 사용
# VLLM 모델 설정
vllm_model = LLM(
    model="./llama3-8b-rag-ko-merged",
    dtype="bfloat16",
    gpu_memory_utilization=0.5   # 0.5(50%), 0.4(40%) 등 환경에 맞게 조정
)

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("./llama3-8b-rag-ko-merged")
print(f"모델의 기본 chat template 확인: {tokenizer.chat_template}")

### Fine-tuned Model from HuggingFaceHub
1. 어댑터모델만 HuggingFaceHub에 업로드해둔 경우, 내려받아서 병합을 진행후, 별도의 병합모델 저장소로 push한다.
   (기반모델로 vLLM객체를 만든후 어댑터를 generate함수 호출시마다 연동해 진행하는 것도 가능)
2. 이미 병합된 모델을 HuggingFaceHub에 업로드해둔 경우 바로 vLLM객체로 생성한다. 

#### 1. 어댑터모델 병합 및 업로드

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# 어댑터 모델 로드
model_id = 'shqkel/llama3-8b-rag-ko'

peft_model = AutoPeftModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)


# 모델 병합: LoRA 등 PEFT 어댑터를 base 모델에 합쳐서 "단일 모델"로 만듬
merged_model = peft_model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained(model_id) 

In [ ]:
# 병합모델 저장소 업로드하기
from huggingface_hub import login 

login(token="your_huggingface_hub_token")

repo_id = "your_huggingface_hub_repo"  # 미리 생성해둘 필요 없음
merged_model.push_to_hub(repo_id, private=False, commit_message="Upload model weights")  
tokenizer.push_to_hub(repo_id, commit_message="Upload tokenizer files") 

#### 2.병합모델로 vLLM객체 생성

In [9]:
# VLLM 모델 설정
# - 방법1: LoRA 어댑터와 병합된 model_id 제공 (저장소에 config.json이 포함되어야 한다.)
# - 방법2: LoRA를 동적 로딩으로 사용. LLM(model=BASE_MODEL_ID, enable_lora=True), vllm_model.generate(lora_request=...)

# 모델 로드
merged_model_id = 'shqkel/llama3-8b-rag-ko-merged'

vllm_model = LLM(
    model=merged_model_id,
    dtype="bfloat16",
    gpu_memory_utilization=0.5   # 0.5(50%), 0.4(40%) 등 환경에 맞게 조정
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(merged_model_id)

INFO 09-02 01:51:28 [utils.py:326] non-default args: {'model': 'shqkel/llama3-8b-rag-ko-merged', 'dtype': 'bfloat16', 'gpu_memory_utilization': 0.5, 'disable_log_stats': True}


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

INFO 09-02 01:51:36 [__init__.py:711] Resolved architecture: LlamaForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 09-02 01:51:36 [__init__.py:2816] Downcasting torch.float32 to torch.bfloat16.
INFO 09-02 01:51:36 [__init__.py:1750] Using max model len 8192
INFO 09-02 01:51:37 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.


...

WARNING 09-02 01:51:41 [__init__.py:2921] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 09-02 01:51:45 [__init__.py:241] Automatically detected platform cuda.
(EngineCore_0 pid=1361) INFO 09-02 01:51:46 [core.py:636] Waiting for init message from front-end.
(EngineCore_0 pid=1361) INFO 09-02 01:51:46 [core.py:74] Initializing a V1 LLM engine (v0.10.1.1) with config: model='shqkel/llama3-8b-rag-ko-merged', speculative_config=None, tokenizer='shqkel/llama3-8b-rag-ko-merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, en

Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  14% Completed | 1/7 [00:00<00:01,  3.22it/s]
Loading safetensors checkpoint shards:  29% Completed | 2/7 [00:01<00:03,  1.66it/s]
Loading safetensors checkpoint shards:  43% Completed | 3/7 [00:02<00:03,  1.33it/s]
Loading safetensors checkpoint shards:  57% Completed | 4/7 [00:02<00:02,  1.25it/s]
Loading safetensors checkpoint shards:  71% Completed | 5/7 [00:03<00:01,  1.18it/s]
Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:04<00:00,  1.17it/s]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:05<00:00,  1.15it/s]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:05<00:00,  1.25it/s]
(EngineCore_0 pid=1361) 


(EngineCore_0 pid=1361) INFO 09-02 01:52:18 [default_loader.py:262] Loading weights took 5.78 seconds
(EngineCore_0 pid=1361) INFO 09-02 01:52:18 [gpu_model_runner.py:2007] Model loading took 14.9596 GiB and 29.576688 seconds
(EngineCore_0 pid=1361) INFO 09-02 01:52:24 [backends.py:548] Using cache directory: /root/.cache/vllm/torch_compile_cache/4fe8a52f1b/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_0 pid=1361) INFO 09-02 01:52:24 [backends.py:559] Dynamo bytecode transform time: 5.57 s
(EngineCore_0 pid=1361) INFO 09-02 01:52:26 [backends.py:194] Cache the graph for dynamic shape for later use
(EngineCore_0 pid=1361) INFO 09-02 01:52:47 [backends.py:215] Compiling a graph for dynamic shape takes 22.85 s
(EngineCore_0 pid=1361) INFO 09-02 01:52:53 [monitor.py:34] torch.compile takes 28.43 s in total
(EngineCore_0 pid=1361) INFO 09-02 01:52:54 [gpu_worker.py:276] Available KV cache memory: 23.41 GiB
(EngineCore_0 pid=1361) INFO 09-02 01:52:54 [kv_cache_utils.py:849] GPU KV c

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:02<00:00, 27.09it/s]


(EngineCore_0 pid=1361) INFO 09-02 01:52:57 [gpu_model_runner.py:2708] Graph capturing finished in 3 secs, took 0.53 GiB
(EngineCore_0 pid=1361) INFO 09-02 01:52:57 [core.py:214] init engine (profile, create kv cache, warmup model) took 38.61 seconds
INFO 09-02 01:52:58 [llm.py:298] Supported_tasks: ['generate']


### vLLM 실행 예시

In [10]:
# apply_chat_template 예시 출력
example_messages = [
    {"role": "system", "content": "당신은 도움이 되는 AI 어시스턴트입니다."},
    {"role": "user", "content": "안녕하세요, 도와주세요."}
]
formatted_example = tokenizer.apply_chat_template(
    example_messages,
    tokenize=False,
    add_generation_prompt=True
)
print("\napply_chat_template 예시:")
print(formatted_example)


apply_chat_template 예시:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 도움이 되는 AI 어시스턴트입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

안녕하세요, 도와주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>




#### 01.검색

In [11]:
# RAG 프롬프트 템플릿 정의
system_prompt = """당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요."""

In [12]:
# 검색 결과 포맷팅 함수
def format_docs(docs):
    doc_items = [f"doc{i+1}: {doc}" for i, doc in enumerate(docs)]
    docs_str = '\n'.join(doc_items)
    return docs_str

In [13]:
# 원본 VLLM을 사용한 RAG 실행 예시
question = "미국의 ICT 정책 중 AI 관련 주요 내용은 무엇인가?"
print("질문:", question)

질문: 미국의 ICT 정책 중 AI 관련 주요 내용은 무엇인가?


In [14]:
# 검색기 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [15]:
# 1. 검색 수행
retrieved_docs = retriever.invoke(question)
print(f"검색된 문서 수: {len(retrieved_docs)}")

검색된 문서 수: 3


In [16]:
# 2. 검색 결과 포맷팅
docs_str = format_docs(retrieved_docs)
print("\n검색 결과 (포맷팅):")
print(docs_str)


검색 결과 (포맷팅):
doc1: page_content='국가별 ICT 시장동향 93. ICT 주요 정책① 미국 인공지능 리더십 장벽 제거* Federal Register, Removing Barriers to American Leadership in Artificial Intelligence, 2025.01.23.
n(주요  내용) 미국  AI 발전  저해 정 책 폐지  및 글로 벌 리더 십 강화  행동 계 획 추진-2025년 1월 23일 백악관에서 발표한 행정명령*으로, 이전 행정부의 행정명령 14110(안전하고 신뢰할 수 있는 AI 개발 및 사용)을 철회하고 해당 명령에 따른 정책과 지침 재검토 지시-AI 시스템에서 이념적 편향과 사회적 의제 배제를 강조하고, 경제 경쟁력과 국가 안보 증진에 초점을 맞춘 정책 방향 설정n(배경) 이전 행정부의 AI 정책 보다 유연한 혁신 환경 조성 필요성 대두-기존 AI 정책이 과도한 규제와 특정 가치 지향적 접근으로 기술 혁신과 경쟁력을 저해한다는 산업계 및 정치적 비판 증가-자유 시장, 세계적 수준의 연구 기관, 기업가 정신을 기반으로 한 미국 AI 혁신의 강점 회복 필요성 인식-중국, 유럽 등 글로벌 경쟁자들의 AI 기술 발전 가속화에 대응하여 미국의 리더십 위치 재확립 요구 증대n(기대 효과) AI 분야 규제 완화를 통한 글로벌 시장 주도권 확보-규제 부담 감소로 AI 분야 민간 기업의 연구개발 및 혁신 활동 가속화 전망-기업들의 AI 기술 투자 및 상용화 확대로 미국 내 AI 산업 성장과 경제적 이익 창출 예상-AI 기술 개발에서 국가 안보와 연계된 전략적 우위 확보를 통해 기술 주권 강화 및 글로벌 표준 설정 영향력 증대표 | 정책 일정 요구사항출처 : Federal Register, Removing Barriers to American Leadership in Artificial Intelligence, 2025.01.23.' metadata={'total_pages': 31, 'page': 8, 'sou

#### 02.증강

In [17]:
# 3. 메시지 준비
user_prompt = f"""질문: {question}

docs:
{docs_str}

위의 docs 중에서만 정보를 근거로 하여, 질문에 답변해 주세요.
답변에서 인용한 문서의 내용에는 반드시 [[doc1]], [[doc2]], ... 형식으로 인용 표시를 해주세요.
추론이나 지어내는 답변은 삼가주시고, docs에 명시적으로 나타난 내용만 인용해 주세요."""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]
print("\n메시지 형식:")
print(f"System: {messages[0]['content']}")
print(f"User: {messages[1]['content']}")


메시지 형식:
System: 당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.
User: 질문: 미국의 ICT 정책 중 AI 관련 주요 내용은 무엇인가?

docs:
doc1: page_content='국가별 ICT 시장동향 93. ICT 주요 정책① 미국 인공지능 리더십 장벽 제거* Federal Register, Removing Barriers to American Leadership in Artificial Intelligence, 2025.01.23.
n(주요  내용) 미국  AI 발전  저해 정 책 폐지  및 글로 벌 리더 십 강화  행동 계 획 추진-2025년 1월 23일 백악관에서 발표한 행정명령*으로, 이전 행정부의 행정명령 14110(안전하고 신뢰할 수 있는 AI 개발 및 사용)을 철회하고 해당 명령에 따른 정책과 지침 재검토 지시-AI 시스템에서 이념적 편향과 사회적 의제 배제를 강조하고, 경제 경쟁력과 국가 안보 증진에 초점을 맞춘 정책 방향 설정n(배경) 이전 행정부의 AI 정책 보다 유연한 혁신 환경 조성 필요성 대두-기존 AI 정책이 과도한 규제와 특정 가치 지향적 접근으로 기술 혁신과 경쟁력을 저해한다는 산업계 및 정치적 비판 증가-자유 시장, 세계적 수준의 연구 기관, 기업가 정신을 기반으로 

In [18]:
# 4. 토크나이저로 프롬프트 포맷팅
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print("\n최종 프롬프트:")
print(prompt)


최종 프롬프트:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 여러 문서(docs)를 바탕으로, 사용자의 질문에 최대한 정확하게, 그리고 문서 내에서만 정보를 근거로 하여 답변하는 AI 비서입니다.
아래 지침을 반드시 지켜주세요:

- 답변은 반드시 docs에서 찾은 내용에 한해서만 작성해주세요. docs에 없는 내용은 추론하거나 지어내지 마세요.
- 답변에서 인용하는 부분이 있다면, 반드시 해당 문서의 번호(예: [[doc1]], [[doc2]])로 근거를 표시해 주세요.
- docs의 순서와 번호는 중요합니다. docs에서 인용하지 않은 정보는 답변에 포함하지 마세요.
- 답변의 근거가 되는 문서 번호를 생략하지 말고, 항상 인용 태그([[doc1]], [[doc2]], ...)를 포함해 주세요.
- 모든 답변은 존댓말을 사용하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

질문: 미국의 ICT 정책 중 AI 관련 주요 내용은 무엇인가?

docs:
doc1: page_content='국가별 ICT 시장동향 93. ICT 주요 정책① 미국 인공지능 리더십 장벽 제거* Federal Register, Removing Barriers to American Leadership in Artificial Intelligence, 2025.01.23.
n(주요  내용) 미국  AI 발전  저해 정 책 폐지  및 글로 벌 리더 십 강화  행동 계 획 추진-2025년 1월 23일 백악관에서 발표한 행정명령*으로, 이전 행정부의 행정명령 14110(안전하고 신뢰할 수 있는 AI 개발 및 사용)을 철회하고 해당 명령에 따른 정책과 지침 재검토 지시-AI 시스템에서 이념적 편향과 사회적 의제 배제를 강조하고, 경제 경쟁력과 국가 안보 증진에 초점을 맞춘 정책 방향 설정n(배경) 이전 행정부의 AI 정책 보다 유연한 혁신 환경 조성 필요성 대두-기존 

#### 03.vLLM기반 생성

In [19]:
# 5. VLLM으로 생성

# 생성 파라미터 설정
sampling_params = SamplingParams(
    temperature=0,
    max_tokens=1024,
)


outputs = vllm_model.generate([prompt], sampling_params)
response = outputs[0].outputs[0].text

# 6. 최종 답변 출력
print("\n최종 답변:")
print(response)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


최종 답변:
미국의 ICT 정책 중 AI 관련 주요 내용은 다음과 같습니다. 

2025년 1월 23일, 백악관에서 발표한 행정명령 "Removing Barriers to American Leadership in Artificial Intelligence"는 AI 리더십 장벽을 제거하고 미국의 AI 발전을 촉진하기 위한 정책을 포함하고 있습니다. 이 행정명령은 이전 행정부의 AI 정책인 "Safe and Secure AI Development and Use"를 철회하고, AI 시스템에서 이념적 편향과 사회적 의제를 배제하는 것을 강조합니다. 또한, 경제 경쟁력과 국가 안보를 중점으로 한 정책 방향을 설정하고 있습니다[[doc1]].

이 정책은 기존의 AI 정책이 과도한 규제와 특정 가치 지향적 접근으로 인해 기술 혁신과 경쟁력을 저해했다는 비판에 부응하여, 자유 시장과 기업가 정신을 기반으로 한 미국의 AI 혁신을 회복하려는 목적을 가지고 있습니다. 결과적으로, 이 정책은 AI 분야의 규제 완화를 통해 글로벌 시장에서의 주도권을 확보하고, AI 기술 개발에서 국가 안보와 연계된 전략적 우위를 확보하는 데 기여할 것으로 예상됩니다[[doc1]].


### RAG 함수화

In [20]:
# 원본 VLLM을 사용한 RAG 함수 구현
def generate_rag_response(question, retriever, tokenizer, vllm_model, sampling_params):
    """
    검색 결과와 질문을 기반으로 RAG 응답을 생성하는 함수
    """
    # 검색 수행
    retrieved_docs = retriever.invoke(question)

    # 검색 결과 포맷팅
    docs_str = format_docs(retrieved_docs)

    # 메시지 준비
    user_prompt = f"""질문: {question}

docs:
{docs_str}

위의 docs 중에서만 정보를 근거로 하여, 질문에 답변해 주세요.
답변에서 인용한 문서의 내용에는 반드시 [[doc1]], [[doc2]], ... 형식으로 인용 표시를 해주세요.
추론이나 지어내는 답변은 삼가주시고, docs에 명시적으로 나타난 내용만 인용해 주세요."""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # 토크나이저로 프롬프트 포맷팅
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # VLLM으로 생성
    outputs = vllm_model.generate([prompt], sampling_params)

    # 응답 추출 및 반환
    response = outputs[0].outputs[0].text
    return response, docs_str, retrieved_docs

In [21]:
# 다른 질문으로 테스트 - 함수 사용
question = "미국의 디지털 인프라 투자에 대해 설명해주세요."
print("\n질문:", question)

# 1. 함수 호출하여 RAG 실행
response, docs_str, retrieved_docs = generate_rag_response(
    question=question,
    retriever=retriever,
    tokenizer=tokenizer,
    vllm_model=vllm_model,
    sampling_params=sampling_params
)

# 2. 결과 출력
print(f"검색된 문서 수: {len(retrieved_docs)}")
print("\n검색 결과:")
print(docs_str)
print("===")
print("\n최종 답변:")
print(response)



질문: 미국의 디지털 인프라 투자에 대해 설명해주세요.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

검색된 문서 수: 3

검색 결과:
doc1: page_content='국가별 ICT 시장동향 113. ICT 주요 정책③ CHIPS  프로그램 개편* The White House, ESTABLISHING THE UNITED STATES INVESTMENT ACCELERATOR, 2025.03.31.  
n(주요 내용) '미국 투자 가속기' 설립을 통한 반도체 제조 투자 가속화 -2025년 3월 31일 백악관에서 발표한 행정명령*으로, 미 상무부 내 '미국 투자 가속기(United States Investment Accelerator)' 신설 지시-10억 달러 이상 투자 촉진을 위한 규제 절차 간소화, 자원 접근성 향상, 국립 연구소와의 협력 증진 등 포괄적 지원 체계 구축-CHIPS 프로그램 사무소를 투자 가속기 산하로 이관하고, 이전 행정부보다 납세자에게 유리한 조건으로 협상 추진 강조n(배경) 글로벌 경쟁 심화와 공급망 문제로 자국 생산 역량 강화 필요성 제기-복잡하고 부담스러운 미국 규제 프로세스가 국내외 기업의 대규모 투자와 반도체 생산시설 구축을 어렵게 한다는 문제 인식-여러 연방, 주, 지방 법규와 중복되는 요구사항으로 인한 건설 지연 및 투자 효율성 저하 문제 해결 필요성 대두-글로벌 반도체 공급망 취약성과 경쟁국의 반도체 산업 지원 정책 확대에 대응하여 국내 생산 기반 확충 시급성 증가n(기대 효과) 반도체 생산 확대를 통한 ICT 산업 경쟁력 강화 -규제 장벽 완화와 정부 지원 체계 강화로 반도체 제조 관련 국내외 기업의 대규모 투자 유치 및 건설 기간 단축 예상-미국 내 반도체 생산 능력 확대를 통한 공급망 회복력 강화 및 첨단 기술 분야 국가 안보 이익 증진-반도체 제조 생태계 발전으로 관련 산업 일자리 창출과 지역 경제 활성화 및 글로벌 기술 경쟁에서 미국의 주도적 위치 강화표 | 미국 투자 가속기 주요 기능출처 : The White House, ESTABLISHING THE UNITED STATES INVESTMENT ACCELERAT